In [1]:
import os
import sys
sys.path.append('../../src')
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
from datetime import datetime
import time
from hydra import initialize, compose
import pathlib
# import config

import data_process.neg_sample as ng_sample
from data_process.utils import mix_merge
from data_process.data_split import data_split_user
from evaluate_entity import CustomHR, CustomNDCG, CustomRoc, CustomRoctop, CustomRecall_top, CustomPrecision_top
from model_entity import EntityCat
from data_utils import CatData
from utils.constants import DEFAULT_USER_COL,DEFAULT_ITEM_COL,DEFAULT_RATING_COL, DEFAULT_TIMESTAMP_COL


from sklearn import metrics, preprocessing
from sklearn.metrics import roc_auc_score, average_precision_score, recall_score, precision_score

import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

# import argparse
torch.manual_seed(0)

/Users/hao/.pyenv/versions/3.6.15/envs/torch-cpu5/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from ignite.engine import Engine, Events, create_supervised_trainer, create_supervised_evaluator, RemovableEventHandle
from ignite.metrics import Accuracy, Loss, Metric
from ignite.handlers import ModelCheckpoint, EarlyStopping
from ignite.exceptions import NotComputableError
from ignite.metrics.metric import sync_all_reduce, reinit__is_reduced
from ignite.contrib.handlers.tqdm_logger import ProgressBar
# from ignite.contrib.handlers import TensorboardLogger 
from ignite.contrib.handlers.wandb_logger import *

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [4]:
with initialize(version_base=None, config_path="../conf"):
    cfg = compose(config_name="config", overrides=[])

In [5]:
if device.type =='cpu':
    BATCH_SIZE = cfg.params.batch_size_cpu
    EPOCHS  = cfg.params.epochs_cpu
else:
    BATCH_SIZE = cfg.params.batch_size_gpu
    EPOCHS  = cfg.params.epochs_gpu

In [6]:
if device.type == 'cpu':
    use_amp=False
    df_train_pos  = ng_sample.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_pos))
    df_train_neg = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_neg))
    df_test_ori = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.test)).iloc[:202,]
    df_all_features = pd.read_csv(pathlib.Path(cfg.path.root, cfg.file.all_features))
    df_train_pos = df_train_pos.sort_values(by=[DEFAULT_USER_COL]).iloc[:100,].reset_index(drop=True)
    df_train_neg = df_train_neg.sort_values(by=[DEFAULT_USER_COL]).iloc[:100*cfg.params.neg_train,].reset_index(drop=True)
else:
    use_amp=True
    df_train_pos  = ng_sample.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_pos))
    df_train_neg = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_neg))
    df_test_ori = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.test))
    df_all_features = pd.read_csv(pathlib.Path(cfg.path.root, cfg.file.all_features))
    df_train_pos = df_train_pos.sort_values(by=[DEFAULT_USER_COL]).reset_index(drop=True)
    df_train_neg = df_train_neg.sort_values(by=[DEFAULT_USER_COL]).reset_index(drop=True)

In [7]:
df_train_pos[DEFAULT_RATING_COL] = 1

In [8]:
def concat_index(df1, df2):
    df2.index = df2.index//cfg.params.neg_train
    return pd.concat([df1, df2], axis=0).sort_index(kind='mregesort').reset_index(drop=True)

In [9]:
df_train_all = concat_index(df_train_pos, df_train_neg)

In [10]:
df_train_all['flag'] = 1
df_test_ori['flag'] = 0
df_all = pd.concat([df_train_all, df_test_ori], axis=0).reset_index(drop=True)

user features: 
       'WindowID_user', 'Split', 'City',
       'State', 'Country', 'Zip_user', 'DegreeType', 'Major', 'GraduationDate',
       'WorkHistoryCount', 'TotalYearsExperience', 'CurrentlyEmployed',
       'ManagedOthers', 'ManagedHowMany',
       
job features: 
       'WindowID_job', 'City_job',
       'State_job', 'Country_job', 'Zip_job', 'StartDate', 'EndDate',

### Choose the features and process data for the training

In [11]:
df_all.columns

Index(['userid', 'itemid', 'rating', 'flag'], dtype='object')

In [39]:
user_features = ['DegreeType', 'Major', 'GraduationDate']
user_features_extend = [DEFAULT_USER_COL] + user_features

item_features = []
item_features_extend =[DEFAULT_ITEM_COL] + item_features

base_features = [DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_RATING_COL]

In [40]:
df_mix_merge = mix_merge(df_all , df_all_features, user_features_extend, item_features_extend)

In [41]:
def _cat_encode(df_data, list_f, encoder):
    for f in list_f:
        df_data[f] = encoder.fit_transform(df_data[f].astype('category').cat.codes.values)
    return df_data

In [42]:
def _embedding_dimension(df_all_encode, features_to_train, max_dim=50):

    embedding_size = []
    features_to_em = [i for i in features_to_train if i !=DEFAULT_RATING_COL]
    for c in features_to_em:
        num_unique_values = int(df_all_encode[c].nunique())
        embed_dim = int(min(np.ceil(num_unique_values/2), max_dim))
        embedding_size.append([num_unique_values, embed_dim])  
    return embedding_size

In [43]:
def encode_data(df_mix_merge, features_to_code, features_to_train, max_dim=50):
    encoder = preprocessing.LabelEncoder()
    df_all_encode = _cat_encode(df_mix_merge, features_to_code, encoder)
    df_train = df_all_encode[df_all.flag==1]
    df_test = df_all_encode[df_all.flag==0]
    df_train = df_train[features_to_train]
    df_test = df_test[features_to_train]
    embedding_size = _embedding_dimension(df_all_encode, features_to_train, max_dim)
    return df_train, df_test, embedding_size

In [44]:
num_feature=[]
features_to_code = df_mix_merge.columns
features_to_train = [DEFAULT_USER_COL, DEFAULT_ITEM_COL]+ user_features + item_features +[DEFAULT_RATING_COL]


In [45]:
df_train,  df_test, embedding_size = encode_data(df_mix_merge, features_to_code, features_to_train, max_dim=cfg.params.emb_dim)

print(f'The size of embedding layers:{embedding_size}')

The size of embedding layers:[[36, 18], [413, 50], [6, 3], [26, 13], [22, 11]]


## Run data check before training 

Check the ratio of positive and negative samples

In [46]:
assert len(df_train[df_train.rating==0])/len(df_train[df_train.rating==1]) == cfg.params.neg_train, 'wrong neg/pos ratio in training set'
assert len(df_test[df_test.rating==0])/len(df_test[df_test.rating==1]) == cfg.params.neg_test, 'wrong neg/pos ratio in test set '
#Check if all the users in test can be found in training set
assert sum(np.isin(df_test.userid.unique(), df_train.userid.unique(), assume_unique=True)) == len(df_test.userid.unique()), 'cold start'
#The the uniqueness of items between training and test. For a user, on common items between training and test dataset. 
assert df_all.shape[0] ==df_train.shape[0]+df_test.shape[0], 'wrong data concat'
assert sum(df_all.groupby(['userid']).apply(lambda x: len(x['itemid'].unique()))) == df_all.shape[0], 'train and test have overlap item'

## Creat the numpy array for training 

In [47]:
df_train_split, df_val_split = data_split_user(df_train, val_size=0.2)

np_train = df_train_split.values
np_val = df_val_split.values
np_test = df_test.values

In [48]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [49]:
train_dataset = CatData(np_train)
val_dataset = CatData(np_val)
test_dataset = CatData(np_test) 
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0,  worker_init_fn=seed_worker,generator=g)
val_loader = data.DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False, num_workers=0,  worker_init_fn=seed_worker,generator=g)
test_loader = data.DataLoader(test_dataset, batch_size=cfg.params.neg_test+1, shuffle=False, num_workers=0,worker_init_fn=seed_worker,generator=g )

In [50]:
model = EntityCat(embedding_size = embedding_size, num_numerical_cols = len(num_feature),
               output_size = 2)
model.to(device)

EntityCat(
  (all_embeddings): ModuleList(
    (0): Embedding(36, 18)
    (1): Embedding(413, 50)
    (2): Embedding(6, 3)
    (3): Embedding(26, 13)
    (4): Embedding(22, 11)
  )
  (mlp_layers): Sequential(
    (0): Linear(in_features=95, out_features=100, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.4, inplace=False)
  )
  (predict_layer): Linear(in_features=100, out_features=2, bias=True)
)

In [51]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=cfg.params.lr)

In [52]:
to_save = {'model': model}
save_path = '/Users/hao/Documents/MA_thesis/ncf-torch2/src/jupyter/runs/'
checkpoint_fp = save_path + "best_model_99_auc=0.6631.pt"
checkpoint = torch.load(checkpoint_fp, map_location=device) 
ModelCheckpoint.load_objects(to_load=to_save, checkpoint=checkpoint) 


RuntimeError: Error(s) in loading state_dict for EntityCat:
	Missing key(s) in state_dict: "mlp_layers.0.weight", "mlp_layers.0.bias", "predict_layer.weight", "predict_layer.bias". 
	Unexpected key(s) in state_dict: "layers.0.weight", "layers.0.bias", "layers.3.weight", "layers.3.bias". 
	size mismatch for all_embeddings.0.weight: copying a param with shape torch.Size([89946, 50]) from checkpoint, the shape in current model is torch.Size([36, 18]).
	size mismatch for all_embeddings.1.weight: copying a param with shape torch.Size([139292, 50]) from checkpoint, the shape in current model is torch.Size([413, 50]).
	size mismatch for all_embeddings.2.weight: copying a param with shape torch.Size([7, 4]) from checkpoint, the shape in current model is torch.Size([6, 3]).
	size mismatch for all_embeddings.3.weight: copying a param with shape torch.Size([14075, 50]) from checkpoint, the shape in current model is torch.Size([26, 13]).
	size mismatch for all_embeddings.4.weight: copying a param 

In [36]:
def output_trans_loss(output):
    return output['y_pred'], output['label']

val_metrics_test = {
    'hr': CustomHR(),
    'ndcg': CustomNDCG(),
    'auc': CustomRoc(),
    'roc_top': CustomRoctop(),
    'recall_top': CustomRecall_top(threshold=0.5),
    'precision_top': CustomPrecision_top(threshold=0.5),
    "loss": Loss(criterion, output_transform=output_trans_loss)
}
def test_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, label = batch[0].to(device), batch[1].to(device)
        y_pred = model(x)[:,1]
        label=label.float()
        y_pred_top, indices = torch.topk(y_pred, engine.state.topk)
        y_pred_top = y_pred_top.detach().cpu().numpy()
        reco_item = torch.take(x[:,1], indices).cpu().numpy().tolist()
        pos_item = x[0,1].cpu().numpy().tolist()  # ground truth, item id
        label_top = label[indices].cpu().numpy()
        return {'pos_item':pos_item, 'reco_item':reco_item, 'y_pred_top':y_pred_top, 'label_top':label_top, 'label':label, 'y_pred':y_pred}

test_evaluator = Engine(test_step)
test_evaluator.state_dict_user_keys.append('topk')
@test_evaluator.on(Events.STARTED)
def init_user_value():
    test_evaluator.state.topk=cfg.params.topk
    
for name, metric in val_metrics_test.items():
    metric.attach(test_evaluator, name)

In [38]:
# @trainer.on(Events.COMPLETED)
# def log_test_results(trainer):
test_evaluator.run(test_loader)
metrics = test_evaluator.state.metrics
hr = metrics['hr']
ndcg = metrics['ndcg']
auc = metrics['auc']
roc_top = metrics['roc_top']
recall = metrics['recall_top']
precision = metrics['precision_top']
loss = metrics['loss']
print(f"Test Results - Avg loss: {loss:.2f} \
 Avg ndcg: {ndcg:.2f}  Avg auc: {auc:.2f}  Avg auc_top: {roc_top:.2f} \
  Avg recall: {recall:.2f}  Avg precision: {precision:.2f}")

Test Results - Avg loss: 0.46  Avg ndcg: 0.16  Avg auc: 0.46  Avg auc_top: 0.11   Avg recall: 0.00  Avg precision: 0.00
